In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '../raw')
# sys.path.clear()

In [2]:
import json, csv
import numpy as np
import pandas as pd

train_df = pd.read_csv(sys.path[0]+('/train.csv'))
train_df[20:40:2]

,id,keyword,location,text,target
20,31,NaN,NaN,this is ridiculous....,0
22,33,NaN,NaN,Love skiing,0
24,36,NaN,NaN,LOOOOOOL,0
26,38,NaN,NaN,Was in NYC last week!,0
28,40,NaN,NaN,Cooool :),0
30,44,NaN,NaN,The end!,0
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
36,54,ablaze,Pretoria,@PhDSquares #mufc they've built so much hype a...,0
38,56,ablaze,NaN,Barbados #Bridgetown JAMAICA ÛÒ Two cars set ...,1


## fill missing values

In [72]:
y = train_df['target']
y.shape

(7613,)

In [73]:
fill_v = {'keyword': 'missing', 'location': 'unknown'}
filled_df = train_df.fillna(value=fill_v)
filled_df = filled_df.drop(['target'], axis=1)

filled_df.insert(4, 'entities','',True)
filled_df.insert(5, 'labels','',True)
filled_df.head(3)

,id,keyword,location,text,entities,labels
0,1,missing,unknown,Our Deeds are the Reason of this #earthquake M...,,
1,4,missing,unknown,Forest fire near La Ronge Sask. Canada,,
2,5,missing,unknown,All residents asked to 'shelter in place' are ...,,


## using `spacy`

In [70]:
import spacy
nlp = spacy.load("en_core_web_sm")

## add entities and labels

In [138]:
from tqdm import tqdm

ent_df = filled_df.copy()
for ri, row in tqdm(ent_df.iterrows()):
    doc = nlp(row['text'])
    ents = []
    labels = []
    for ent in doc.ents:
#         print(ent.text)
        if ent.text not in ents:
            ents.append(ent.text)
        if ent.label_ not in labels:
            labels.append(ent.label_)
            
    ent_df.at[ri, 'entities'] = ','.join(ents)
    ent_df.at[ri, 'labels'] = ','.join(labels)
        
#     ent_df.at[ri, 'entities'] += row['entities'] + ent.text+ ","
#     ent_df.at[ri, 'labels'] += row['labels'] + ent.label_+ ","
#     print()
#     if ri > 5:
#         break

7613it [01:14, 101.97it/s]


In [137]:
ent_df.head(5)

,id,keyword,location,text,entities,labels
0,1,missing,unknown,Our Deeds are the Reason of this #earthquake M...,Deeds,PERSON
1,4,missing,unknown,Forest fire near La Ronge Sask. Canada,"La Ronge Sask,Canada","FAC,GPE"
2,5,missing,unknown,All residents asked to 'shelter in place' are ...,,
3,6,missing,unknown,"13,000 people receive #wildfires evacuation or...","13,000,California","CARDINAL,GPE"
4,7,missing,unknown,Just got sent this photo from Ruby #Alaska as ...,Ruby #Alaska,PERSON


In [142]:
nlp_df = ent_df.copy()
nlp_df.insert(6, 'ent_dep', '', True)
nlp_df.insert(7, 'ent_head', '', True)
nlp_df.insert(8, 'ent_pos', '', True)
nlp_df.insert(9, 'ent_children', '', True)
nlp_df.head(5)

,id,keyword,location,text,entities,labels,ent_dep,ent_head,ent_pos,ent_children
0,1,missing,unknown,Our Deeds are the Reason of this #earthquake M...,Deeds,PERSON,,,,
1,4,missing,unknown,Forest fire near La Ronge Sask. Canada,"La Ronge Sask,Canada","FAC,GPE",,,,
2,5,missing,unknown,All residents asked to 'shelter in place' are ...,,,,,,
3,6,missing,unknown,"13,000 people receive #wildfires evacuation or...","13,000,California","CARDINAL,GPE",,,,
4,7,missing,unknown,Just got sent this photo from Ruby #Alaska as ...,Ruby #Alaska,PERSON,,,,


In [151]:
for ri, row in nlp_df.iterrows():
    doc = nlp(row['text'])
    entities = [ent_tok.lower() for ent_tok in row['entities'].split(',') if ent_tok]
#     print(entities)
    deps = []
    heads =[]
    positions = [] 
    children = set()
    
    for token in doc:
        if token.text.lower() in entities:
#             print(token.text.lower())
            if token.dep_ not in deps:
                deps.append(token.dep_)
            if token.head.text not in heads:
                heads.append(token.head.text)
            if token.head.pos_ not in positions:
                positions.append(token.head.pos_)
                
            children = children.union([child.text for child in token.children])
            
#             nlp_df.at[ri, 'ent_dep'] = row['ent_dep'] + token.dep_ + ","
#             nlp_df.at[ri, 'ent_head'] += row['ent_head'] + token.head.text + ","
#             nlp_df.at[ri, 'ent_pos'] += row['ent_pos'] + token.head.pos_ + ","
#             nlp_df.at[ri, 'ent_children'] += row['ent_children'] + ','.join([child.text for child in token.children])+','
    nlp_df.at[ri, 'ent_dep'] = ','.join(deps)
    nlp_df.at[ri, 'ent_head'] = ','.join(heads)
    nlp_df.at[ri, 'ent_pos'] = ','.join(positions)
    nlp_df.at[ri, 'ent_children'] = ','.join(children)
            
#     if ri > 5:
#         break


In [152]:
nlp_df.head(15)

,id,keyword,location,text,entities,labels,ent_dep,ent_head,ent_pos,ent_children
0,1,missing,unknown,Our Deeds are the Reason of this #earthquake M...,Deeds,PERSON,nsubj,are,AUX,Our
1,4,missing,unknown,Forest fire near La Ronge Sask. Canada,"La Ronge Sask,Canada","FAC,GPE",ROOT,Canada,PROPN,
2,5,missing,unknown,All residents asked to 'shelter in place' are ...,,,,,,
3,6,missing,unknown,"13,000 people receive #wildfires evacuation or...","13,000,California","CARDINAL,GPE",pobj,in,ADP,
4,7,missing,unknown,Just got sent this photo from Ruby #Alaska as ...,Ruby #Alaska,PERSON,,,,
5,8,missing,unknown,#RockyFire Update => California Hwy. 20 closed...,"RockyFire,California Hwy,20,Lake County","PERSON,ORG,CARDINAL,GPE","compound,nummod","Update,.","PROPN,PUNCT",
6,10,missing,unknown,#flood #disaster Heavy rain causes flash flood...,"Manitou,Colorado Springs",GPE,pobj,in,ADP,
7,13,missing,unknown,I'm on top of the hill and I can see a fire in...,,,,,,
8,14,missing,unknown,There's an emergency evacuation happening now ...,,,,,,
9,15,missing,unknown,I'm afraid that the tornado is coming to our a...,,,,,,


In [153]:
X = nlp_df.copy()

X.join(y).to_csv('../datasets/entities_nlp_train_hongyu.csv')
X.shape

(7613, 10)

## training

In [8]:
df = pd.read_csv('../datasets/entities_nlp_train_hongyu.csv', index_col=0)
df.head(5)

,id,keyword,location,text,entities,labels,ent_dep,ent_head,ent_pos,ent_children,target
0,1,missing,unknown,Our Deeds are the Reason of this #earthquake M...,Deeds,PERSON,nsubj,are,AUX,Our,1
1,4,missing,unknown,Forest fire near La Ronge Sask. Canada,"La Ronge Sask,Canada","FAC,GPE",ROOT,Canada,PROPN,NaN,1
2,5,missing,unknown,All residents asked to 'shelter in place' are ...,NaN,NaN,NaN,NaN,NaN,NaN,1
3,6,missing,unknown,"13,000 people receive #wildfires evacuation or...","13,000,California","CARDINAL,GPE",pobj,in,ADP,NaN,1
4,7,missing,unknown,Just got sent this photo from Ruby #Alaska as ...,Ruby #Alaska,PERSON,NaN,NaN,NaN,NaN,1


In [14]:
fillers = {'entities':'None', 'labels':'None', 'ent_dep':'None', 'ent_head':'None', 'ent_pos':'None', 'ent_children':'None'}
df = df.fillna(value=fillers)
df.to_csv('../datasets/entities_nlp_train_hongyu.csv', index=False)
df.head(5)

,id,keyword,location,text,entities,labels,ent_dep,ent_head,ent_pos,ent_children,target
0,1,missing,unknown,Our Deeds are the Reason of this #earthquake M...,Deeds,PERSON,nsubj,are,AUX,Our,1
1,4,missing,unknown,Forest fire near La Ronge Sask. Canada,"La Ronge Sask,Canada","FAC,GPE",ROOT,Canada,PROPN,None,1
2,5,missing,unknown,All residents asked to 'shelter in place' are ...,None,None,None,None,None,None,1
3,6,missing,unknown,"13,000 people receive #wildfires evacuation or...","13,000,California","CARDINAL,GPE",pobj,in,ADP,None,1
4,7,missing,unknown,Just got sent this photo from Ruby #Alaska as ...,Ruby #Alaska,PERSON,None,None,None,None,1


In [15]:
y = df['target']
y.shape

(7613,)

In [16]:
X = df.drop(['target'], axis=1)
X.columns

Index(['id', 'keyword', 'location', 'text', 'entities', 'labels', 'ent_dep',
       'ent_head', 'ent_pos', 'ent_children'],
      dtype='object')

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorizer = CountVectorizer()
vectorizer = CountVectorizer(ngram_range=(1, 1), analyzer='word')
x = vectorizer.fit_transform(X['labels'][:10])
# x.toarray()
vectorizer.get_feature_names()

['cardinal', 'fac', 'gpe', 'none', 'org', 'person']

In [37]:
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer

class LabeledNormalizer(Normalizer):
    def fit(self, X, *args, **kwargs):
        try:
            self.names = X.columns
        except:
            self.names = [str(i) for i in range(X.shape[1])]
        return super().fit(X, *args, **kwargs)
         
    def get_feature_names(self):
        return self.names
    
vec = ColumnTransformer([
#     ('norm', LabeledNormalizer(), ['id']),
    ('kw', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"(?u)\b\w+\b",), 'keyword'),
    ('loc', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"(?u)\b\w+\b"), 'location'),
    ('text', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"(?u)\b\w+\b"), 'text'),
    ('ent', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'entities'),
    ('label', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'labels'),
    ('dep', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_dep'),
    ('head', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_head'),
    ('pos', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_pos'),
    ('child', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_children'),
    
])

from sklearn.svm import LinearSVC
from sklearn.svm import SVC
clf = SVC(kernel='linear', probability=True) # 1.
# clf = SVC(kernel='rbf', probability=True) # 2.
# clf = LinearSVC(verbose=True)


pipeline = make_pipeline(vec, clf)

## evaluation

### 1. hyper`C`=1, `kernel`=linear, `gamma`=scale

In [30]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict

def evaluate(_clf, X, y):
    report = classification_report(
        y_true=y, y_pred=cross_val_predict(pipeline, X, y, cv=5)
    )
    print(report)
    scores = cross_val_score(_clf, X, y, scoring='accuracy', cv=10)
    print('Cross-validation Accuracy: {:.3f} ± {:.3f}'.format(np.mean(scores), 2 * np.std(scores)))
    
    _clf.fit(X,y)
    print('Training Set Accuracy: {:.3f}'.format(_clf.score(X,y)))


evaluate(pipeline, X, y)

              precision    recall  f1-score   support

           0       0.65      0.68      0.67      4342
           1       0.55      0.52      0.54      3271

    accuracy                           0.61      7613
   macro avg       0.60      0.60      0.60      7613
weighted avg       0.61      0.61      0.61      7613

Cross-validation Accuracy: 0.516 ± 0.197
Training Set Accuracy: 0.975


In [38]:
from sklearn.externals import joblib
import os

pipeline.fit(X,y)
entity_mdl = '../models/nlp-entity-linear-svc.pkl'
joblib.dump(pipeline, entity_mdl)
!ls -lSh $entity_mdl

-rw-r--r-- 1 hongyu hongyu 8.2M Mar 17 21:54 ../models/nlp-entity-linear-svc.pkl


### 2. hyper`C`=10, `kernel`=rbf, `gamma`=auto

### 2. `kernel`=rbf

In [35]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict

def evaluate(_clf, X, y):
    report = classification_report(
        y_true=y, y_pred=cross_val_predict(pipeline, X, y, cv=5)
    )
    print(report)
    scores = cross_val_score(_clf, X, y, scoring='accuracy', cv=10)
    print('Cross-validation Accuracy: {:.3f} ± {:.3f}'.format(np.mean(scores), 2 * np.std(scores)))
    
    _clf.fit(X,y)
    print('Training Set Accuracy: {:.3f}'.format(_clf.score(X,y)))


evaluate(pipeline, X, y)

KeyboardInterrupt: 